In [1]:
from __future__ import absolute_import,division,print_function,unicode_literals
import numpy as np
import matplotlib.pyplot as plt
from  IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc 
import tensorflow as tf
import pandas as pd
import os # accessing directory structure
import random
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model



In [2]:
for dirname, _, filenames in os.walk("../input/dataset-test"):
    files=[]
    for filename in filenames:
        print(os.path.join(dirname, filename))
        files.append([os.path.join(dirname, filename)])

../input/dataset-test/dataset3.csv


In [28]:
#read dataset
data = pd.read_csv("../input/dataset-test/dataset3.csv",sep=",", header=None)
d=[data[i] for i in range(2)]
f=pd.DataFrame(d, index=["text","state"])
f.index
data=f.T
#change the result value
data['state'] =data['state'].replace(['normal','attack'],[0,1])

#split data totest and train 
train_df,test_df = train_test_split(data, test_size=0.25, random_state=42)
# train_res=train_df.pop("state")
# test_res=test_df.pop("state")


In [21]:
test_df

,text,state
4003,"/index.php None */* gzip,deflate close http://...",1
4635,"/index.php None */* gzip,deflate close http://...",1
8583,/_/visualfrontendui/browserinfo gh1.1.29749278...,0
5418,/_/visualfrontendui/browserinfo gh1.1.29749278...,0
7301,/_/visualfrontendui/browserinfo gh1.1.29749278...,0
...,...,...
7372,/_/visualfrontendui/browserinfo gh1.1.29749278...,0
1405,"/index.php None */* gzip,deflate no-cache clos...",1
2625,"/index.php None */* gzip,deflate close http://...",1
7338,/_/visualfrontendui/browserinfo gh1.1.29749278...,0


In [29]:
# convert string to lower case
train_texts = train_df["text"].values
train_texts = [s.lower() for s in train_texts]
test_texts = test_df["text"].values
test_texts = [s.lower() for s in test_texts]
# =======================Convert string to index================
# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)



In [ ]:
print(len(train_res))

In [30]:
# Convert string to index
train_sequences = tk.texts_to_sequences(train_texts)
test_sequences = tk.texts_to_sequences(test_texts)

# Padding
train_data = pad_sequences(train_sequences, maxlen=1014, padding='post')
test_data  = pad_sequences(test_sequences, maxlen=1014, padding='post')

# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')



In [31]:
train_data

array([[ 8., 44.,  8., ...,  0.,  0.,  0.],
       [ 8.,  4.,  6., ...,  0.,  0.,  0.],
       [ 8., 44.,  8., ...,  0.,  0.,  0.],
       ...,
       [ 8., 44.,  8., ...,  0.,  0.,  0.],
       [ 8.,  4.,  6., ...,  0.,  0.,  0.],
       [ 8., 44.,  8., ...,  0.,  0.,  0.]], dtype=float32)

In [42]:
train_classes = train_df["state"].values
train_class_list = []
for x in train_classes:
    if x == 0 :
        train_class_list.append(0)
    else:
        train_class_list.append(1)

test_classes = test_df["state"].values
test_class_list = []
for x in test_classes:
    if x == 0 :
        test_class_list.append(0)
    else:
        test_class_list.append(1)
        
from keras.utils import to_categorical
train_classes = to_categorical(train_class_list)
test_classes = to_categorical(test_class_list)

In [74]:
# =====================Char CNN=======================
# parameter
input_size = 1014
vocab_size = len(tk.word_index)
embedding_size = 69
conv_layers = [[256, 7, 3],
               [256, 7, 3],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
num_of_classes = 2
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'

# Embedding weights
embedding_weights = []  # (70, 69)
embedding_weights.append(np.zeros(vocab_size))  # (0, 69)

for char, i in tk.word_index.items():  # from index 1 to 69
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)

embedding_weights = np.array(embedding_weights)
# Embedding layer Initialization
embedding_layer = Embedding(vocab_size + 1,
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])



In [77]:
# Model Construction
# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)

# Embedding
x = embedding_layer(inputs)

# Conv
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x)
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x)  # Final shape=(None, 34, 256)
x = Flatten()(x)  # (None, 8704)

# Fully connected layers
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x)  # dense_size == 1024
    x = Dropout(dropout_p)(x)

    # Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)

# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])  # Adam, categorical_crossentropy
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 1014)]            0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 1014, 69)          4830      
_________________________________________________________________
conv1d_96 (Conv1D)           (None, 1008, 256)         123904    
_________________________________________________________________
activation_96 (Activation)   (None, 1008, 256)         0         
_________________________________________________________________
max_pooling1d_48 (MaxPooling (None, 336, 256)          0         
_________________________________________________________________
conv1d_97 (Conv1D)           (None, 330, 256)          459008    
_________________________________________________________________
activation_97 (Activation)   (None, 330, 256)          0     

In [ ]:
# Shuffle

indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)


x_train = train_data[indices]
y_train = train_classes[indices]

x_test = test_data
y_test = test_classes


# # Training
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=10,
          verbose=2)

Epoch 1/10
